In [1]:
from aoc import read_file

In [302]:
lines = read_file('20221207_test.txt').split('\n')

commands = [line.split(' ') for line in lines]

In [189]:
import json

def parse_commands(commands):
    i = 1

    root = {
        'name': '/',
        'path': '/',
        'type': 'dir',
        'contents': {}
    }

    dir_stack = [root]

    current_dir = root

    while i < len(lines):
        command = commands[i]
        if command[0] != '$':
            raise Exception('Should see command: {lines[index]}')
        if command[1] == 'cd':
            if command[2] == '..':
                current_dir = dir_stack.pop()
            else:
                directory_name = command[2]
                dir_stack.append(current_dir)
                current_dir = current_dir['contents'][directory_name]
        elif command[1] == 'ls':
            while i+1 < len(commands) and commands[i+1][0] != '$':
                i += 1
                if commands[i][0] == 'dir':
                    directory_name = commands[i][1]
                    current_dir['contents'][directory_name] = {
                        'name': directory_name,
                        'path': current_dir['path'] + '/' + directory_name,
                        'type': 'dir',
                        'contents': {}
                    }
                else:
                    size = int(commands[i][0])
                    file_name = commands[i][1]
                    current_dir['contents'][file_name] = {
                        'name': file_name,
                        'type': 'file',
                        'size': size
                    }
        i += 1

    return root

In [203]:
directory_tree = parse_commands(commands)

In [220]:
def get_directory_sizes(directory):
    sizes = {
        'name': directory['name'],
        'path': directory['path'],
        'contents': {}
    }
    
    dir_size = 0
    
    for name, content_file in directory['contents'].items():
        if content_file['type'] == 'file':
            dir_size += content_file['size']
        else:
            subdir_size = get_directory_sizes(content_file)
            sizes['contents'][name] = subdir_size
            dir_size += subdir_size['size']

    sizes['size'] = dir_size
    return sizes

In [248]:
directory_sizes = get_directory_sizes(directory_tree)

In [249]:
def get_directory_size_map(directory_sizes):
    ret = {
        directory_sizes['path']: directory_sizes['size']
    }
    for sub_dir in directory_sizes['contents'].values():
        ret = {**ret, **get_directory_size_map2(sub_dir)}
    return ret

In [251]:
size_map = get_directory_size_map2(directory_sizes)

In [252]:
sum([size for path, size in size_map.items() if size <= 100000])

1491614

In [299]:
available_space = 40000000

used_space = size_map['/']

required_space = used_space - available_space

for size in sorted([size for path, size in size_map.items()]):
    if size < required_space:
        continue
    else:
        print(size)
        break


6400111


# Alternative version, inspired by Peter's solution

In [270]:
# The function is called ls_r as an analogy to the bash command ls -r *
def ls_r(commands):
    stack = []
    for command in commands:
        if command[0] == '$':
            if command[1] == 'cd':
                if command[2] == '..':
                    stack.pop()
                else:
                    stack.append(command[2])
            # Don't care about catching ls
        elif command[0] == 'dir':
            pass
        else:
            # This is a file, so save the full path along with the file size
            yield (' '.join(stack), int(command[0]))


In [271]:
list(ls_r(commands))

[('/', 14848514),
 ('/', 8504156),
 ('/ a', 29116),
 ('/ a', 2557),
 ('/ a', 62596),
 ('/ a e', 584),
 ('/ d', 4060174),
 ('/ d', 8033020),
 ('/ d', 5626152),
 ('/ d', 7214296)]

In [311]:
def expand_path(path):
    path_split = path.split(' ')
    for i in range(len(path_split)):
        yield ' '.join(path_split[:i+1])

In [312]:
[
    (expanded_path, size)
    for path, size in ls_r(commands)
    for expanded_path in expand_path(path)
]

[('/', 14848514),
 ('/', 8504156),
 ('/', 29116),
 ('/ a', 29116),
 ('/', 2557),
 ('/ a', 2557),
 ('/', 62596),
 ('/ a', 62596),
 ('/', 584),
 ('/ a', 584),
 ('/ a e', 584),
 ('/', 4060174),
 ('/ d', 4060174),
 ('/', 8033020),
 ('/ d', 8033020),
 ('/', 5626152),
 ('/ d', 5626152),
 ('/', 7214296),
 ('/ d', 7214296)]

In [307]:
from collections import Counter

def aggregate_sizes(path_sizes):
    c = Counter()
    for expanded_path, size in path_sizes:
        c[expanded_path] += size
    return c


In [308]:
file_sizes = aggregate_sizes((
    (expanded_path, size)
    for path, size in ls_r(commands)
    for expanded_path in expand_path(path)
))

file_sizes

Counter({'/': 48381165, '/ a': 94853, '/ a e': 584, '/ d': 24933642})

In [309]:
sum([size for size in file_sizes.values() if size <= 100000])

95437

In [310]:
available_space = 40000000

used_space = file_sizes['/']

required_space = used_space - available_space

for size in sorted([size for size in file_sizes.values()]):
    if size < required_space:
        continue
    else:
        print(size)
        break


24933642
